#### Load data about patients diagnosed with vascular dementia

In [ ]:
UKBB_PATH = "~/biobank/ukb672220.csv"

fields_dict = {
    "id":"eid",
    "Age":"34-0.0",
    "Sex":"31-0.0",
    "Primary Hypertension":"131287-0.0",
    "Secondary Hypertension":"131286-0.0",
    "Vascular Dementia Date First Reported":"130838-0.0",
    "Date of vascular dementia report" : "42022-0.0",
    "BMI":"21001-2.0",
    "Smoking Status":"20116-0.0",
    "Ever Smoked":"20160-0.0",
    "Alcohol Frequency": "20414-0.0",
    "Report of stroke": "42007-0.0",
    "Recent restlessness": "20516-0.0",
}

In [ ]:
bloodcount_list = [
    "30160-0.0", "30160-0.1", "30160-0.2",
    "30220-0.0", "30220-0.1", "30220-0.2",
    "30150-0.0", "30150-0.1", "30150-0.2",
    "30210-0.0", "30210-0.1", "30210-0.2",
    "30030-0.0", "30030-0.1", "30030-0.2",
    "30020-0.0", "30020-0.1", "30020-0.2",
    "30300-0.0", "30300-0.1", "30300-0.2",
    "30290-0.0", "30290-0.1", "30290-0.2",
    "30280-0.0", "30280-0.1", "30280-0.2",
    "30120-0.0", "30120-0.1", "30120-0.2",
    "30180-0.0", "30180-0.1", "30180-0.2",
    "30050-0.0", "30050-0.1", "30050-0.2",
    "30060-0.0", "30060-0.1", "30060-0.2",
    "30040-0.0", "30040-0.1", "30040-0.2",
    "30100-0.0", "30100-0.1", "30100-0.2",
    "30260-0.0", "30260-0.1", "30260-0.2",
    "30270-0.0", "30270-0.1", "30270-0.2",
    "30130-0.0", "30130-0.1", "30130-0.2",
    "30190-0.0", "30190-0.1", "30190-0.2",
    "30140-0.0", "30140-0.1", "30140-0.2",
    "30200-0.0", "30200-0.1", "30200-0.2",
    "30170-0.0", "30170-0.1", "30170-0.2",
    "30230-0.0", "30230-0.1", "30230-0.2",
    "30080-0.0", "30080-0.1", "30080-0.2",
    "30090-0.0", "30090-0.1", "30090-0.2",
    "30110-0.0", "30110-0.1", "30110-0.2",
    "30010-0.0", "30010-0.1", "30010-0.2",
    "30070-0.0", "30070-0.1", "30070-0.2",
    "30250-0.0", "30250-0.1", "30250-0.2",
    "30240-0.0", "30240-0.1", "30240-0.2",
    "30000-0.0", "30000-0.1", "30000-0.2"
]
bloodchem_list = [
    "30620-0.0", "30620-0.1",
    "30600-0.0", "30600-0.1",
    "30610-0.0", "30610-0.1",
    "30630-0.0", "30630-0.1",
    "30640-0.0", "30640-0.1",
    "30650-0.0", "30650-0.1",
    "30710-0.0", "30710-0.1",
    "30680-0.0", "30680-0.1",
    "30690-0.0", "30690-0.1",
    "30700-0.0", "30700-0.1",
    "30720-0.0", "30720-0.1",
    "30730-0.0", "30730-0.1",
    "30740-0.0", "30740-0.1",
    "30750-0.0", "30750-0.1",
    "30770-0.0", "30770-0.1",
    "30760-0.0", "30760-0.1",
    "30780-0.0", "30780-0.1",
    "30790-0.0", "30790-0.1",
    "30800-0.0", "30800-0.1",
    "30810-0.0", "30810-0.1",
    "30820-0.0", "30820-0.1",
    "30830-0.0", "30830-0.1",
    "30840-0.0", "30840-0.1",
    "30850-0.0", "30850-0.1",
    "30860-0.0", "30860-0.1",
    "30870-0.0", "30870-0.1",
    "30880-0.0", "30880-0.1",
    "30670-0.0", "30670-0.1",
    "30890-0.0", "30890-0.1"
]
ecg = [
    "12336-0.2", "12336-0.3",
    "12338-0.2", "12338-0.3",
    "22334-0.2", "22334-0.3",
    "22330-0.2", "22330-0.3",
    "22338-0.2", "22338-0.3",
    "12340-0.2", "12340-0.3",
    "22331-0.2", "22331-0.3",
    "22332-0.2", "22332-0.3",
    "22333-0.2", "22333-0.3",
    "22335-0.2", "22335-0.3",
    "22336-0.2", "22336-0.3",
    "22337-0.2", "22337-0.3"
]

blood_pressure = [
    "4080-0.0", "4080-0.1", "4080-0.2", "4080-0.3", # Systolic
    "4079-0.0", "4079-0.1", "4079-0.2", "4079-0.3", # Diastolic
    "102-0.0", "102-0.1", "102-0.2", "102-0.3", # Pulse Rate
]

mental_health = [
    "20544-0.1",
    "20544-0.2",
    "20544-0.3",
    "20544-0.4",
    "20544-0.5",
    "20544-0.6",
    "20544-0.7",
    "20544-0.8",
    "20544-0.9",
    "20544-0.10",
    "20544-0.11",
    "20544-0.12",
    "20544-0.13",
    "20544-0.14",
    "20544-0.15",
    "20544-0.16"
]


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [ ]:
columns = pd.read_csv(UKBB_PATH, nrows=0).columns.tolist()
print(columns)

In [ ]:
fields = list(fields_dict.values())
vd_1 = fields_dict["Vascular Dementia Date First Reported"]
vd_2 = fields_dict["Date of vascular dementia report"]
chunk_size = 20000

filtered_chunks = []
counter = 0
print("Working..")

for chunk in pd.read_csv(UKBB_PATH, usecols=fields, chunksize=chunk_size):
    filtered_chunk = chunk[chunk[vd_1].notna() | chunk[vd_2].notna()]

    filtered_chunk = filtered_chunk.copy()
    filtered_chunk["VascularDementiaReportDate"] = filtered_chunk[vd_1].combine_first(filtered_chunk[vd_2])

    # Convert "VascularDementia" column to datetime
    filtered_chunk["VascularDementiaReportDate"] = pd.to_datetime(filtered_chunk["VascularDementiaReportDate"], errors="coerce")
    
    # Create the "hasVascularDementia" column
    filtered_chunk["hasVascularDementia"] = filtered_chunk["VascularDementiaReportDate"].apply(
                lambda x: 1 if pd.Timestamp("1950-01-01") <= x <= pd.Timestamp("2030-12-31") else 0
                )

    filtered_chunks.append(filtered_chunk)
    
    counter += len(filtered_chunk)
    print(counter)

df = pd.concat(filtered_chunks, ignore_index=True)

df.to_pickle('vascular_dementia_filtered.pkl')
df.head(30)

#2181

In [ ]:
fields = list(fields_dict.values())
vd_1 = fields_dict["Vascular Dementia Date First Reported"]
vd_2 = fields_dict["Date of vascular dementia report"]
chunk_size = 20000

counter = 0
print("Working...")

# Create an empty DataFrame to hold the filtered data
result_df = pd.DataFrame()

for chunk in pd.read_csv(UKBB_PATH, usecols=fields, chunksize=chunk_size, low_memory=False):
    # Filter rows where either vd_1 or vd_2 is not NaN
    filtered_chunk = chunk[chunk[vd_1].notna() | chunk[vd_2].notna()].copy()  # Make a copy here
    
    # Combine dates and add the new columns
    filtered_chunk.loc[:, "VascularDementiaReportDate"] = pd.to_datetime(
        filtered_chunk[vd_1].combine_first(filtered_chunk[vd_2]), errors="coerce"
    )
    filtered_chunk.loc[:, "hasVascularDementia"] = (
        filtered_chunk["VascularDementiaReportDate"]
        .between(pd.Timestamp("1950-01-01"), pd.Timestamp("2030-12-31"))
        .astype(int)
    )
    
    # Append to the result DataFrame
    result_df = pd.concat([result_df, filtered_chunk], ignore_index=True)
    
    # Update the counter
    counter += len(filtered_chunk)
    print(counter)

# Save the result
result_df.to_pickle('vascular_dementia_filtered.pkl')

# Display the first 30 rows
result_df.head(30)